### Import modules

In [1]:
import keras_bitcoin
import dataset
import collections
import numpy as np
import tfidf

Using TensorFlow backend.


In [ ]:
#poolling

### Create Train/Test Dataset

In [3]:
df = dataset.get_labeled_dataset(number_of_file=5, from_date="2015-12-16")
print(collections.Counter(df["label"]))
print(df["text"])

Number of files loaded :  5
Counter({1: 124, 0: 76})
0      Might be more factual if you said they were fi...
1      My opinion is this. Reddit is built to filter ...
2      It is indeed impressive how thin-skinned some ...
3      Welcome to Reddit /u/Coloco.\r\n\r\n\r\nOdd ho...
4      Roger's intentions are (probably/hopefully) pu...
5      >contradicting his words with his actions\r\n\...
6      It's just the result of principled idealism. I...
7      In my opinion btcdrak is unsuitable to be a fo...
8      You must be new if you think /u/memorydealers ...
9      Don't spend it all in one place. It might be w...
10     Hot damn, it's the most valuable quarter I eve...
11     Yes, but I don't see how it relates to it bein...
12                                   /r/BTC_CessPool lol
13     Let me give some examples of the moderation st...
14     Blocked a post by Gavin (at least I suspect he...
15           We could all go and restart /r/WeAreBitcoin
16                                 

In [21]:
#LDA
df = dataset.get_LDA_data()
df.head()

,Unnamed: 0,day,result,label
0,0,2015-12-15,glad moderation http community wwwredditcom d...,1
1,1,2015-12-16,glad furthermore moderation subreddit communi...,0
2,2,2015-12-17,develop function fashbinder_pwn similar power...,1
3,3,2015-12-30,subreddit block ad money hear think cs away p...,0
4,4,2015-12-31,glad holiday http ha hold wwwredditcom fund u...,1


In [7]:
#TFIDF
df_idf = dataset.get_TFIDF_data()
labels_dic = dataset.get_labeled_bitcoin_price()
df_idf.head()
labels = []
for date in df_idf["date"]:
    labels.append(labels_dic[date])
df_idf["label"] = labels
df_idf.head()

,Unnamed: 0,date,words,label
0,0,2015-12-15,"['yes', 'mod', 'bullshit', 'btc', 'post', 'lol...",1
1,1,2015-12-16,"['bitcoin', 'btc', 'borg', 'play', 'well', 'do...",0
2,2,2015-12-17,"['downvote', 'ignore', 'someone', 'similar', '...",1
3,3,2015-12-30,"['reddit', 'ad', 'away', 'block', 'body', 'css...",0
4,4,2015-12-31,"['changetip', 'ads', 'lol', 'theymos', 'nice',...",1


### Format Dataset

In [5]:
texts = df["text"]
labels = df["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size = keras_bitcoin.get_train_test_data(texts, labels)

In [22]:
#LDA
texts = df["result"]
labels = df["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size = keras_bitcoin.get_train_test_data(texts, labels)

In [9]:
#IDF
texts = df_idf["words"]
labels = df_idf["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size = keras_bitcoin.get_train_test_data(texts, labels)

### Set Model Params

In [16]:

epochs=20
batch_size=None
activations_functions=["sigmoid", "relu"]
dropouts = {}

### Train Mode

In [17]:
model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length, max_sentence_size, epochs=epochs, batch_size=batch_size, activations_functions=activations_functions, verbose=1)

C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
534/534 [==============================] - 0s 494us/step - loss: 0.7808 - acc: 0.4494
Epoch 2/20
534/534 [==============================] - 0s 148us/step - loss: 0.7320 - acc: 0.4494
Epoch 3/20
534/534 [==============================] - 0s 145us/step - loss: 0.6930 - acc: 0.4494
Epoch 4/20
534/534 [==============================] - 0s 127us/step - loss: 0.6603 - acc: 0.5599
Epoch 5/20
534/534 [==============================] - 0s 133us/step - loss: 0.6272 - acc: 0.8914
Epoch 6/20
534/534 [==============================] - 0s 129us/step - loss: 0.5946 - acc: 0.9382
Epoch 7/20
534/534 [==============================] - 0s 124us/step - loss: 0.5589 - acc: 0.9288
Epoch 8/20
534/534 [==============================] - 0s 120us/step - loss: 0.5193 - acc: 0.9345
Epoch 9/20
534/534 [==============================] - 0s 116us/step - loss: 0.4738 - acc: 0.9551
Epoch 10/20
534/534 [==============================] - 0s 110us/step - loss: 0.4254 - acc: 0.9738
Epoch 11/20
534/534 [========

###  Model Evaluation

In [18]:
loss, accuracy = model.evaluate(texts_test, labels_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

134/134 [==============================] - 0s 463us/step
Accuracy: 52.238804


###  Get Predictions

In [20]:
df_test = dataset.get_labeled_dataset(number_of_file = 10, from_date = "2017-02-10", date_included = False, all_files=True)
print(df_test.shape,df_test.head(2))

Number of files loaded :  668
(117152, 3)                                                 text  label        date
0                                               body      1  2015-12-15
1  Recent comment by him with regards to Gavin An...      1  2015-12-15


In [21]:
preds = keras_bitcoin.get_predictions(list(df_test["text"]), model, vocab_length, max_sentence_size)
df_test["preds"] = list(map(lambda x : int(x), preds))
df_test["correct"] = np.equal(preds, df_test["label"])

### Predictions Stats

In [22]:
dataset.get_prediction_stats(df_test)

Number Correct/Wrong Guess : 18267/98885
              Accuracy : 15.592563507238461
Invalid sentences count 82195


### Find best settings

In [25]:
def get_combinations(arr):
    combinations = []
    for i in range(len(arr)):
        item = arr[i]
        for j in range(i + 1, len(arr)):
            combinations.append([item, arr[j]])
    return combinations

In [30]:
import json
import pandas as pd

epochs=10
batch_size=None
perfs = {}
perfs["epochs"] = epochs
perfs["batch_size"] = batch_size
perfs["number_files_used"] = 12
dropout = {}

function_list = get_combinations(keras_bitcoin.available_activation_functions)
#function_list = keras_bitcoin.available_activation_functions
#function_list = [["softplus", "relu", "exponential", "softmax"]]
for function in function_list:
    
    if not isinstance(function, list):
        function = [function]
    else :
        function = list(function)
        dropout[function[0]] = 0.1
        #dropout[function[2]] = 0.1
    model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length, max_sentence_size, epochs=epochs, batch_size=batch_size, activations_functions=function, verbose = 0)
    loss, accuracy = model.evaluate(texts_test, labels_test, verbose=0)
    perfs["_".join(function)] = [accuracy]
#with open(f"./keras_stats/ep={epochs}_bats={batch_size}.json", "w") as fp:
#    json.dump(perfs, fp)

C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_128 (Embedding)    (None, 100, 20)           2600      
_________________________________________________________________
flatten_128 (Flatten)        (None, 2000)              0         
_________________________________________________________________
dense_244 (Dense)            (None, 1)                 2001      
_________________________________________________________________
dense_245 (Dense)            (None, 1)                 2         
Total params: 4,603
Trainable params: 4,603
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_129"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_129 (Embedding)    (None, 100, 20)           2600      
_________________________________________________________________
flatten_129 (Flatten)        (None, 2000)              0         
_________________________________________________________________
dense_246 (Dense)            (None, 1)                 2001      
_________________________________________________________________
dense_247 (Dense)            (None, 1)                 2         
Total params: 4,603
Trainable params: 4,603
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Model: "sequential_180"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_180 (Embedding)    (None, 100, 20)           2600      
_________________________________________________________________
flatten_180 (Flatten)        (None, 2000)              0         
_________________________________________________________________
dense_348 (Dense)            (None, 1)                 2001      
_________________________________________________________________
dense_349 (Dense)            (None, 1)                 2         
Total params: 4,603
Trainable params: 4,603
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_181"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_181 (Embedding)    (None, 100, 20)           2600      
_________________________________________________________________
flatten_181 (Flatten)        (None, 2000)              0         
_________________________________________________________________
dense_350 (Dense)            (None, 1)                 2001      
_________________________________________________________________
dense_351 (Dense)            (None, 1)                 2         
Total params: 4,603
Trainable params: 4,603
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_182"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_182 (Embedding)    (None, 100, 20)           2600      
_________________________________________________________________
flatten_182 (Flatten)        (None, 2000)              0         
_________________________________________________________________
dense_352 (Dense)            (None, 1)                 2001      
_________________________________________________________________
dense_353 (Dense)            (None, 1)                 2         
Total params: 4,603
Trainable params: 4,603
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
perfs

{'epochs': 10,
 'batch_size': None,
 'number_files_used': 12,
 'tanh_elu': [1.0],
 'tanh_softmax': [0.6521106362342834],
 'tanh_selu': [1.0],
 'tanh_softplus': [1.0],
 'tanh_softsign': [0.34788936376571655],
 'tanh_relu': [0.34788936376571655],
 'tanh_sigmoid': [1.0],
 'tanh_hard_sigmoid': [1.0],
 'tanh_exponential': [0.7918485999107361],
 'tanh_linear': [1.0],
 'elu_softmax': [0.6521106362342834],
 'elu_selu': [1.0],
 'elu_softplus': [1.0],
 'elu_softsign': [0.9985443949699402],
 'elu_relu': [1.0],
 'elu_sigmoid': [1.0],
 'elu_hard_sigmoid': [1.0],
 'elu_exponential': [0.45414847135543823],
 'elu_linear': [0.34788936376571655],
 'softmax_selu': [0.0],
 'softmax_softplus': [0.6521106362342834],
 'softmax_softsign': [0.0],
 'softmax_relu': [0.0],
 'softmax_sigmoid': [0.6521106362342834],
 'softmax_hard_sigmoid': [0.6521106362342834],
 'softmax_exponential': [0.6521106362342834],
 'softmax_linear': [0.6521106362342834],
 'selu_softplus': [0.4497816562652588],
 'selu_softsign': [0.6564774

In [107]:
import os
import pandas as pd

directory = "./keras_stats/"
dicts = []
for file in os.listdir(directory):
    f = open(f'{directory}{file}')
    dicts.append(json.load(f))

functions = keras_bitcoin.available_activation_functions

perfs_stats = {"function" : [], "min" : [], "max" : [], "avg" : []}
for function in functions:
    sum_perfs = 0
    min_perfs = 1
    max_perfs = -1
    for dct in dicts:
        value = dct[function]
        sum_perfs += value
        if value > max_perfs:
            max_perfs = value
        if value < min_perfs:
            min_perfs = value
    perfs_stats["function"].append(function)
    perfs_stats["avg"].append(sum_perfs / len(functions))
    perfs_stats["max"].append(max_perfs)
    perfs_stats["min"].append(min_perfs)

print(pd.DataFrame(perfs_stats))
#softmax softplus softsign sigmoid hard_sigmoid  

        function       min       max       avg
0           tanh  0.000000  0.619942  0.382144
1            elu  0.000000  0.589487  0.054955
2        softmax  0.617438  0.617438  0.449046
3           selu  0.000000  0.622862  0.267266
4       softplus  0.415102  0.624948  0.409944
5       softsign  0.574885  0.617856  0.441006
6           relu  0.000000  0.622028  0.161262
7        sigmoid  0.600334  0.626617  0.445102
8   hard_sigmoid  0.592407  0.619942  0.445102
9    exponential  0.000000  0.624531  0.320514
10        linear  0.000000  0.618273  0.208367


In [59]:
functions = ["softmax", "softplus", "softsign", "sigmoid", "hard_sigmoid"]

In [40]:
import json
from IPython.display import clear_output

epochs = 8
batch_size = None
perfs = {}
perfs["number_files_used"] = 12
functions_list = [["tanh","exponential"]]
dropout_value = 0.2
for functions in functions_list:
    functions = list(functions)
    dropouts = {}
    #dropouts[functions[0]] = 0.1
    #dropouts[functions[1]] = 0.08
    model = keras_bitcoin.get_model(texts_train = texts_train, labels_train = labels_train, vocab_length = vocab_length, max_sentence_size = max_sentence_size, epochs = epochs, 
    batch_size = batch_size, activations_functions = functions, verbose = 0, dropouts = dropouts)
    loss, accuracy = model.evaluate(texts_test, labels_test, verbose=0)
    #perfs["_".join(functions)] = accuracy
    clear_output()
    print("acc : ", accuracy)


acc :  0.8165938854217529


In [96]:
last_date = "2010-10-11"
for i in range(200):
    df_test = dataset.get_labeled_dataset(number_of_file = 5, from_date = last_date, date_included = False)
    preds = keras_bitcoin.get_predictions(list(df_test["text"]), model, vocab_length, max_sentence_size)
    df_test["preds"] = list(map(lambda x : int(x), preds))
    df_test["correct"] = np.equal(preds, df_test["label"])
    #dataset.get_prediction_stats(df_test)
    last_date = list(df_test["date"])[-1]

Number Correct/Wrong Guess : 2938/448
              Accuracy : 86.7690490253987
Invalid sentences count 0
Number Correct/Wrong Guess : 0/2560
              Accuracy : 0.0
Invalid sentences count 0
Number Correct/Wrong Guess : 3020/1403
              Accuracy : 68.27944833823197
Invalid sentences count 0
Number Correct/Wrong Guess : 1319/880
              Accuracy : 59.9818099135971
Invalid sentences count 0
Number Correct/Wrong Guess : 853/1376
              Accuracy : 38.26828174069089
Invalid sentences count 0
Number Correct/Wrong Guess : 1038/611
              Accuracy : 62.94724075197089
Invalid sentences count 0
Number Correct/Wrong Guess : 693/574
              Accuracy : 54.69613259668509
Invalid sentences count 0
Number Correct/Wrong Guess : 1418/332
              Accuracy : 81.02857142857142
Invalid sentences count 0
Number Correct/Wrong Guess : 702/1022
              Accuracy : 40.71925754060325
Invalid sentences count 0
Number Correct/Wrong Guess : 722/515
              Accu

ZeroDivisionError: division by zero

In [ ]:
 functions = ["softmax", "softplus", "softsign", "sigmoid", "hard_sigmoid"]
[["softplus", "sigmoid"], ["softsign", "sigmoid"], ["softsign", "hard_sigmoid"], ["sigmoid", "hard_sigmoid"]]

In [7]:
df_idf = tfidf.get_tfidf_words(df, 100)

3431/3431


In [8]:
df_idf["tfidf"] = [" ".join(text) for text in df_idf["tfidf"]]
df_idf.head()

,text,label,date,score,nb_replies,stickied,tfidf
0,might factual said firework sound wor explo te...,0,2015-12-16,-4,0,False,actual ad agr also back believ bitcoin block b...
1,opinion reddit built filter unpopular opinion ...,0,2015-12-16,15,0,False,actual ad agr also back believ bitcoin block b...
2,ind impress thin skin mod seem,0,2015-12-16,1,0,False,actual ad agr also back believ bitcoin block b...
3,welcom reddit u coloco odd account attack thro...,0,2015-12-16,1,1,False,actual ad agr also back believ bitcoin block b...
4,roger intent probabl hope pure untru made plen...,0,2015-12-16,3,0,False,actual ad agr also back believ bitcoin block b...


In [9]:
texts = df_idf["tfidf"]
labels = df_idf["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size = keras_bitcoin.get_train_test_data(texts, labels)

In [4]:
epochs=5
batch_size=None
activations_functions=["sigmoid", "softsign"]
model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length, max_sentence_size, epochs=epochs, batch_size=batch_size, activations_functions=activations_functions, verbose=0)

NameError: name 'texts_train' is not defined

In [21]:
loss, accuracy = model.evaluate(texts_test, labels_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

687/687 [==============================] - 0s 144us/step
Accuracy: 100.000000
